## Deploying PD, LGD, EAD models to calculate ECL

In this notebook, we import the PD, LGD, and EAD models we developed in the first three notebooks to calculate the Expected Credit Loss (ECL):

$$ECL = PD \times LGD \times EAD$$

Importing the models will be done using the **pickle** package.

Let us import the libraries as well as the datasets for the three models.

In [77]:
# Importing the needed libraries
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm

sns.set()

## Importing and Matching Datasets

For LGD and EAD, we only need to import the pre-processed raw datasets. For PD however, we need to import the dataset comprised of dummy variables we coarse-classed with Weight-of-Evidence calculations.

In [78]:
# Loading the data for LGD and EAD
df_loans = pd.read_csv('loans_preprocessed.csv')

# Loading the dataset for PD calculation
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')

As the dataset for PD is not only split, but also shuffled, we need to combine it and re-arrange it by the original index number in order for the data order to match that of LGD and EAD data.

In [79]:
PD_data = pd.concat([X_train, X_test], axis = 0) # axis is 0 because we are combining rows
PD_data = PD_data.set_index('Unnamed: 0') # Set the old index before exporting into index
PD_data = PD_data.sort_index() # Sort data by new index to match LGD dataset

<br> 

## Loading and Running the Models

We load our PD, LGD, and EAD models using the **pickle** package.

In [80]:
import pickle

# Loading the two PD model
with open('PD_model.pkl', 'rb') as f:
    PD_model = pickle.load(f)


# Loading the two LGD models
with open('RR_model.pkl', 'rb') as f:
    RR_model_1 = pickle.load(f)

with open('RR_model.pkl', 'rb') as f:
    RR_model_2 = pickle.load(f)

# Loading the two EAD_to_LoanAmt model
with open('EAD_model.pkl', 'rb') as f:
    EAD_model = pickle.load(f)

<br>

#### Running the PD model

As with the PD modelling exercise, we need to drop reference categories and also add the intercept constant. We also need to drop dummy variables that were found to have no variance in the modelling phase.

In [81]:
ref_categories = ['Grade:_A', 'home_ownership:_MORTGAGE', 'addr_state:_AL', 'verif:_Not Verified', 'purpose:_cred-card', 
                  'init_list_status:_f', 'term:_36', 'empLength: _(-inf, 0.0]', 'mths_sinc_issue:_(-inf, 0.0]', 
                  'int_rate: _(5, 10]', 'annual_inc: _0k-20k', 'DtI: _(8, 12]', 'mths_last_record: _(0.0, 70.0]']

categ_no_variance = ['empLength: _(10.0, inf]', 'mths_last_record: _(130.0, inf]']


PD_data_with_const = PD_data.drop(ref_categories + ['id'] + categ_no_variance, axis = 1)
PD_data_with_const.insert(0, 'intercept', 1)
PD_data_with_const['intercept'] = PD_data_with_const.intercept.astype(bool)

In [82]:
PD_data_with_const['PD'] = PD_model.predict(PD_data_with_const)

In [83]:
PD_data_with_const['PD'].describe()

count    466285.000000
mean          0.109210
std           0.069475
min           0.007092
25%           0.057082
50%           0.093949
75%           0.145580
max           0.660239
Name: PD, dtype: float64

<br>

#### Running the LGD model

For the LGD and EAD model:
- Our model used a specific set of predictors which we need to filter out,
- We need to add additional variables we added for our model like months since last payment,
- We need to watch whether additional dummies exist that were not in the training data.

In [84]:
# Create a new variable containing the number of months between last_payment_date and end of 2014

# Convert to datetime (the given year is two-digit and needs "20" prefix)
df_loans['prop_last_pymnt_d'] = pd.to_datetime(df_loans['last_pymnt_d'], format='%b-%y')

# Reference date is end of 2016
ref_date = pd.to_datetime('2016-12-31')

# Calculating the difference between ref-date and last payment date in months
df_loans['last_payment_months'] = (ref_date.year - df_loans['prop_last_pymnt_d'].dt.year) * 12 + (ref_date.month - df_loans['prop_last_pymnt_d'].dt.month)


In [85]:
grade_dummies = pd.get_dummies(df_loans.grade, drop_first = True, prefix = 'Grade: ')
home_own_dummies = pd.get_dummies(df_loans.home_ownership, drop_first = True, prefix = 'Home_Own: ')
verif_dummies = pd.get_dummies(df_loans.verification_status, drop_first = True, prefix = 'Verif_Status: ')

# Drop additional dummy column that is not in the original model
home_own_dummies = home_own_dummies.drop('Home_Own: _MORTGAGE', axis = 1)

# Build a new dataframe with only selected features during the modeling exercise
LGD_data = df_loans[['dti','funded_amnt', 'emp_length_int', 'term_int', 'last_payment_months',
                  'mths_since_issue_d', 'mths_since_earliest_cr_line', 'mths_since_last_delinq', 'mths_since_last_record']]

# Combining the selected variables with the dummies
LGD_data = pd.concat([LGD_data, grade_dummies, home_own_dummies, verif_dummies], axis = 1)

# Adding a constant for the intercept
LGD_data.insert(0, 'intercept', 1)

LGD_data = LGD_data.astype(float)

In our modeling exercise, we modeled RR in a two staged approach:
- The first was to find out whether the RR is zero or not,
- The second was to estimate RR when it is not zero.

We got the outcomes of the two models by simple multiplication.

In [86]:
LGD_data['RR'] = RR_model_1.predict(LGD_data) * RR_model_2.predict(LGD_data)

Now we ensure that our RR is capped by ceiled by 1 and floored by 0.

In [87]:
LGD_data['RR'] = np.where(LGD_data['RR'] > 1, 1, LGD_data['RR']) # capping with 1
LGD_data['RR'] = np.where(LGD_data['RR'] < 0, 0, LGD_data['RR']) # floor with 0

LGD_data['RR'].describe()

count    465909.000000
mean          0.224808
std           0.140297
min           0.027092
25%           0.133383
50%           0.182091
75%           0.265713
max           1.000000
Name: RR, dtype: float64

<br>

Finally, we calculate LGD as (1 - Recovery Rate)

In [88]:
LGD_data['LGD'] = 1- LGD_data['RR']

<br>

#### Running the EAD model

The EAD model will use exactly the same approach we used to build the LGD dataset.

In [89]:
# Create a dataframe with model variables for EAD prediction
df_EADtLA = df_loans[['dti','funded_amnt', 'emp_length_int', 'term_int', 'last_payment_months',
                  'mths_since_issue_d', 'mths_since_earliest_cr_line', 'mths_since_last_delinq', 'mths_since_last_record']]

# Combining the dataframe with the dummies
df_EADtLA = pd.concat([df_EADtLA, grade_dummies, home_own_dummies, verif_dummies], axis = 1)

# Adding the intercept term
df_EADtLA.insert(0, 'intercept', 1)

df_EADtLA = df_EADtLA.astype(float)

Our model predicts the ratio of EAD_to_LoanAmount where:

EAD = EAD_to_loanAmt_Ratio * Funded_Amount

In [90]:
df_EADtLA['EAD'] = EAD_model.predict(df_EADtLA)* df_EADtLA['funded_amnt']

In [91]:
df_EADtLA['EAD'].describe()

count    465909.000000
mean       8623.412424
std        6750.774875
min          27.046344
25%        3363.904911
50%        6793.658014
75%       12212.351937
max       34100.283372
Name: EAD, dtype: float64

<br>

### Calculating Expected Loss

Finally, we combine the predicted outcomes from the three models to calculate ECL

In [92]:
df_ECL = pd.concat([df_EADtLA['EAD'], LGD_data['LGD'], PD_data_with_const['PD']], axis = 1)

In [94]:
# Calculating ECL
df_ECL['ECL'] = df_ECL.PD * df_ECL.LGD * df_ECL.EAD

df_ECL.head()

,EAD,LGD,PD,ECL
0,648.558584,0.735352,0.165949,79.144211
1,2088.752571,0.495362,0.196100,202.902774
2,612.660205,0.605356,0.174473,64.708036
3,1361.896480,0.677487,0.179584,165.696639
4,301.308581,0.777915,0.111307,26.089466


In [103]:
# The total expected loss at the portfolio level
total_ECL = df_ECL['ECL'].sum()
print(f'Total Expected Loss: {total_ECL:,.0f}')

total_funded_amount = df_loans['funded_amnt'].sum()
print(f'Percentage of Expected Loss to Total Portfolio is: {total_ECL/total_funded_amount:,.4f}')

Total Expected Loss: 320,718,386
Percentage of Expected Loss to Total Portfolio is: 0.0481
